In [22]:
from google.colab import files
uploaded=files.upload()
uploaded=files.upload()
uploaded=files.upload()


Saving orders.csv to orders (5).csv


Saving users.json to users (2).json


Saving restaurants.sql to restaurants (1).sql


In [24]:
import pandas as pd
orders=pd.read_csv("orders.csv")
orders.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [26]:
users=pd.read_json("users.json")
users.head()

,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [27]:
import sqlite3

conn = sqlite3.connect(":memory:")

with open("restaurants.sql", "r") as f:
    conn.executescript(f.read())

restaurants = pd.read_sql("SELECT * FROM restaurants", conn)
restaurants.head()

,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [28]:
df = orders.merge(users, on="user_id", how="left")
df = df.merge(restaurants, on="restaurant_id", how="left")

df.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [29]:
df.to_csv("final_food_delivery_dataset.csv", index=False)

In [30]:
files.download("final_food_delivery_dataset.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [31]:
import pandas as pd
df = pd.read_csv("final_food_delivery_dataset.csv")

In [32]:
q1 = (
    df[df["membership"]=="Gold"]
    .groupby("city")["total_amount"]
    .sum()
    .idxmax()
)

print("Answer:", q1)

Answer: Chennai


In [33]:
q2 = (
    df.groupby("cuisine")["total_amount"]
    .mean()
    .idxmax()
)

print("Answer:", q2)

Answer: Mexican


In [34]:
user_spend = df.groupby("user_id")["total_amount"].sum()

count = (user_spend > 1000).sum()

print("Number of users:", count)

Number of users: 2544


In [36]:
bins = [3.0, 3.5, 4.0, 4.5, 5.0]
labels = ["3.0–3.5", "3.6–4.0", "4.1–4.5", "4.6–5.0"]

df["rating_range"] = pd.cut(df["rating"], bins=bins, labels=labels)

In [37]:
q4= (
    df.groupby("rating_range")["total_amount"]
    .sum()
    .idxmax()
)

print("Answer:", q4)

Answer: 4.6–5.0


/tmp/ipython-input-3726730938.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby("rating_range")["total_amount"]


In [39]:
q2 = (
    df[df["membership"]=="Gold"]
    .groupby("city")["total_amount"]
    .mean()
    .idxmax()
)

print("Answer:", q2)

Answer: Chennai


In [40]:
rest_count = df.groupby("cuisine")["restaurant_id"].nunique()

In [41]:
revenue = df.groupby("cuisine")["total_amount"].sum()

In [42]:
result = pd.concat([rest_count, revenue], axis=1)
result.columns = ["restaurant_count", "revenue"]

print(result)

         restaurant_count     revenue
cuisine                              
Chinese               120  1930504.65
Indian                126  1971412.58
Italian               126  2024203.80
Mexican               128  2085503.09


In [43]:
print(df.groupby("cuisine")["total_amount"].sum().sort_values(ascending=False))

cuisine
Mexican    2085503.09
Italian    2024203.80
Indian     1971412.58
Chinese    1930504.65
Name: total_amount, dtype: float64


In [44]:
gold_pct = (df[df["membership"]=="Gold"].shape[0] / df.shape[0]) * 100
print(round(gold_pct))

50


In [48]:
print(df.columns)

Index(['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount',
       'restaurant_name_x', 'name', 'city', 'membership', 'restaurant_name_y',
       'cuisine', 'rating', 'rating_range'],
      dtype='object')


In [50]:
df[["restaurant_name_x","restaurant_name_y"]].head()

,restaurant_name_x,restaurant_name_y
0,New Foods Chinese,Restaurant_450
1,Ruchi Curry House Multicuisine,Restaurant_309
2,Spice Kitchen Punjabi,Restaurant_107
3,Darbar Kitchen Non-Veg,Restaurant_224
4,Royal Eatery South Indian,Restaurant_293


In [53]:
rest_stats = df.groupby("restaurant_name_x").agg(
    avg_value=("total_amount","mean"),
    orders=("restaurant_name_x","count")
)

# Sort by average order value
ans = rest_stats.sort_values("avg_value", ascending=False)

print(ans.head(10))  # top 10 restaurants

                                     avg_value  orders
restaurant_name_x                                     
Classic Biryani Chinese            1126.520000      26
Hotel Dhaba Multicuisine           1040.222308      13
Sri Mess Punjabi                   1029.180833      12
Ruchi Biryani Punjabi              1002.140625      16
Sri Delights Pure Veg               989.467222      18
Amma Eatery Non-Veg                 985.054103      39
Sai Restaurant Family Restaurant    982.951818      22
Taste of Cafe Andhra                976.589000      20
Classic Kitchen Family Restaurant   973.167895      19
Hotel Dhaba Chinese                 973.125556      18


In [54]:
rest_stats[rest_stats["orders"] < 20]

,avg_value,orders
restaurant_name_x,,
Amma Biryani South Indian,708.708667,15
Amma Cafe North Indian,667.979091,11
Amma Cafe Pure Veg,839.633529,17
Amma Curry House Family Restaurant,821.800556,18
Amma Delights Family Restaurant,761.249412,17
...,...,...
Udupi Mess Family Restaurant,831.023125,16
Udupi Mess North Indian,898.528125,16
Udupi Restaurant Family Restaurant,796.961053,19


In [55]:

combo = df.groupby(["membership","cuisine"])["total_amount"] \
          .sum() \
          .sort_values(ascending=False)

top_combo = combo.head(1)
print("Answer (Highest revenue combination):", f"{top_combo.index[0][0]} + {top_combo.index[0][1]}")

Answer (Highest revenue combination): Regular + Mexican


In [56]:
import pandas as pd

# Load your final dataset
df = pd.read_csv("final_food_delivery_dataset.csv")

# ---------------- Q1: % of orders by Gold members ----------------
gold_pct = (df[df["membership"]=="Gold"].shape[0] / df.shape[0]) * 100
gold_pct_rounded = round(gold_pct)

# ---------------- Q2: Restaurant with highest avg order value <20 orders ----------------
rest_stats = df.groupby("restaurant_name_x").agg(
    avg_value=("total_amount","mean"),
    orders=("restaurant_name_x","count")
)
ans_restaurant = rest_stats[rest_stats["orders"]<20] \
                    .sort_values("avg_value", ascending=False) \
                    .head(1)

# ---------------- Q3: Highest revenue combination (membership + cuisine) ----------------
combo = df.groupby(["membership","cuisine"])["total_amount"] \
          .sum() \
          .sort_values(ascending=False)
top_combo = combo.head(1)

# ---------------- Q4: Quarter with highest revenue ----------------
df["order_date"] = pd.to_datetime(df["order_date"])
df["quarter"] = df["order_date"].dt.quarter
quarter_revenue = df.groupby("quarter")["total_amount"].sum()
highest_quarter = quarter_revenue.idxmax()
quarter_map = {1:"Q1 (Jan–Mar)", 2:"Q2 (Apr–Jun)", 3:"Q3 (Jul–Sep)", 4:"Q4 (Oct–Dec)"}

# ---------------- Print all answers ----------------
print("Answer Q1 (Percentage of orders by Gold members):", f"{gold_pct_rounded}%")
print("Answer Q2 (Restaurant with highest avg order value <20 orders):", ans_restaurant.index[0])
print("Answer Q3 (Highest revenue combination):", f"{top_combo.index[0][0]} + {top_combo.index[0][1]}")
print("Answer Q4 (Quarter with highest revenue):", quarter_map[highest_quarter])

Answer Q1 (Percentage of orders by Gold members): 50%
Answer Q2 (Restaurant with highest avg order value <20 orders): Hotel Dhaba Multicuisine
Answer Q3 (Highest revenue combination): Regular + Mexican
Answer Q4 (Quarter with highest revenue): Q3 (Jul–Sep)


/tmp/ipython-input-2089232730.py:26: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df["order_date"] = pd.to_datetime(df["order_date"])


In [57]:

q1_total_orders_gold = df[df["membership"]=="Gold"].shape[0]
print("Total orders by Gold members:", q1_total_orders_gold)

Total orders by Gold members: 4987


In [58]:

q2_hyderabad_revenue = round(df[df["city"]=="Hyderabad"]["total_amount"].sum())
print("Total revenue from Hyderabad:", q2_hyderabad_revenue)

Total revenue from Hyderabad: 1889367


In [59]:
q3_distinct_users = df["user_id"].nunique()
print("Distinct users with at least one order:", q3_distinct_users)

Distinct users with at least one order: 2883


In [60]:

q4_avg_order_gold = round(df[df["membership"]=="Gold"]["total_amount"].mean(), 2)
print("Average order value for Gold members:", q4_avg_order_gold)

Average order value for Gold members: 797.15


In [61]:

q5_orders_high_rating = df[df["rating"] >= 4.5].shape[0]
print("Orders for restaurants with rating ≥ 4.5:", q5_orders_high_rating)

Orders for restaurants with rating ≥ 4.5: 3374


In [62]:

gold_df = df[df["membership"]=="Gold"]
top_city = gold_df.groupby("city")["total_amount"].sum().idxmax()
q6_orders_top_city_gold = gold_df[gold_df["city"]==top_city].shape[0]
print("Orders in top revenue city among Gold members:", q6_orders_top_city_gold)

Orders in top revenue city among Gold members: 1337


In [63]:

q1 = "user_id"
print("Column used to join orders.csv and users.json:", q1)

Column used to join orders.csv and users.json: user_id


In [64]:

q2 = "restaurants.sql"
print("Dataset containing cuisine and rating info:", q2)

Dataset containing cuisine and rating info: restaurants.sql


In [65]:

import pandas as pd
df = pd.read_csv("final_food_delivery_dataset.csv")
q3 = df.shape[0]
print("Total number of rows in final merged dataset:", q3)

Total number of rows in final merged dataset: 10000


In [66]:

q4 = "NaN"
print("Values if a user has no matching record in users.json:", q4)

Values if a user has no matching record in users.json: NaN


In [67]:

q5 = "merge()"
print("Pandas function used to combine datasets based on a key:", q5)

Pandas function used to combine datasets based on a key: merge()


In [68]:

q6 = "users.json"
print("Column 'membership' originates from:", q6)

Column 'membership' originates from: users.json


In [69]:

q7 = "restaurant_id"
print("Join key used to combine orders data with restaurant details:", q7)

Join key used to combine orders data with restaurant details: restaurant_id


In [70]:

q8 = "cuisine"
print("Column that helps identify the type of food served by a restaurant:", q8)

Column that helps identify the type of food served by a restaurant: cuisine


In [71]:

q9 = "multiple"
print("If a user places multiple orders, their personal details appear:", q9)

If a user places multiple orders, their personal details appear: multiple
